In [1]:
import tensorflow_datasets as tfds
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os 
from tensorflow import keras
from tensorflow.keras import layers
import pickle
import tensorflow_hub as hub
import progressbar


%matplotlib inline  


In [2]:
#从TFHUB下载模型需要代理
os.environ['https_proxy']='http://192.168.0.164:10809'
os.environ['http_proxy']='http://192.168.0.164:10809'

In [3]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)

In [4]:
((ds_train_imgnet,ds_val_imgnet),ds_info_imgnet) = tfds.load('imagenet2012', split=['train','validation'],shuffle_files=False,with_info=True,as_supervised=True)

In [5]:
#tfds.as_dataframe(ds_val_imgnet.take(10),ds_info_imgnet)

In [6]:
def data_preprocess_imgnet(x,label):
    x = tf.image.convert_image_dtype(x, tf.float32)
    x = tf.image.resize(x,(224,224))
    return (x,label+1)
    
ds_train_imgnet = ds_train_imgnet.map(data_preprocess_imgnet)
ds_val_imgnet  = ds_val_imgnet.map(data_preprocess_imgnet)


In [7]:
AUTOTUNE = tf.data.AUTOTUNE
BATCH_SIZE=256*2
ds_train_imgnet = ds_train_imgnet.prefetch(buffer_size=AUTOTUNE).batch(BATCH_SIZE)
ds_val_imgnet = ds_val_imgnet.prefetch(buffer_size=AUTOTUNE).batch(BATCH_SIZE)


In [8]:
d1=ds_val_imgnet.take(10)

In [9]:

def create_model():
    model = tf.keras.Sequential([
    hub.KerasLayer("https://tfhub.dev/google/imagenet/resnet_v2_50/classification/5")
    ])
    model.build([None, 224, 224, 3])  # Batch input shape.
    model.compile(optimizer='adam',
                    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                    metrics=['accuracy'])
    return model

#strategy = tf.distribute.MirroredStrategy()
#with strategy.scope():

model = create_model()    
    

In [10]:
def do_pred(ds,name_pred,name_bg,PG_FILE='pgfile.p'):
 
  pred_file = open(name_pred,'wb')
  bg_file = open(name_bg,'wb')
  
  ds = ds.enumerate()
  n=0
  try:
    for (i,element) in ds.as_numpy_iterator():
        #y_hat=model.predict(element[0])
        y_hat=model.predict_on_batch(element[0])
        y= element[1]
        pickle.dump(y_hat,pred_file)
        pickle.dump(y,bg_file)
        n=n+1
  except E as N:
    print(N)
    print('except n=',n)
    pred_file.close()
    bg_file.close()


生成验证集上的结果
大约需要3分钟

In [14]:
do_pred(ds_val_imgnet,'imgnet_val_pred.p','imgnet_val_bg.p')

生成训练上的结果

大约需要75分钟

In [12]:
do_pred(ds_train_imgnet,'imgnet_train_pred.p','imgnet_train_bg.p')

In [13]:
print('val_size=',ds_val_imgnet.cardinality(),'train_size=',ds_train_imgnet.cardinality())

val_size= tf.Tensor(98, shape=(), dtype=int64) train_size= tf.Tensor(2503, shape=(), dtype=int64)


In [1]:
def load_ds(filename):
    with open(filename,'rb')  as f:
        a=pickle.load(f)
        a=np.array(a)
        sz=np.shape(a)
        last=sz[0]
        
        while True:
            try:
                b=pickle.load(f)
                szb=np.shape(b)
                if szb[0]+last>sz[0]:
                    if len(szb) == 1:
                        sz=(sz[0]*2,)
                    else:
                        sz=(sz[0]*2,sz[1])
                    a.resize(sz)
                a[last:last+szb[0]]=b[0:szb[0]]
                last=last+szb[0]
            except:
                break
            
        if len(sz)==2:
            sz=(last,sz[1])
        else:
            sz=(last,)
        a.resize(sz)
    return a